# Ensemble 

In [48]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score,KFold
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from mlxtend.regressor import StackingRegressor
import xgboost as xgb

import warnings


warnings.filterwarnings("ignore")

# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

In [49]:
X = train_df.loc[:,'Id':'SaleCondition_Partial']
y = train_df['SalePrice']
print("Shape of training set {}.\nShape of test set {}".format(X.shape,y.shape))

Shape of training set (1460, 303).
Shape of test set (1460,)


In [55]:
lasso = Lasso(alpha=0.0005)
ENet = ElasticNet(alpha=0.001,l1_ratio=.45)
# GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.03,
#                                    max_depth=3, max_features='sqrt', 
#                                    loss='huber', random_state =5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
lr = LinearRegression()


stregr = StackingRegressor(regressors=[lasso,ENet,model_xgb,GBoost], 
                           meta_regressor=lr)
kf = KFold(10, shuffle=True, random_state=42).get_n_splits(X)
rmse_cv_stack= np.sqrt(-cross_val_score(model_xgb, X, y, scoring="neg_mean_squared_error", cv = kf))
print("The 10-fold crossvalidation RMSE of Stack Regressor is {:.5f} +/- {:.3f}".format(rmse_cv_stack.mean(),
                                                                                          rmse_cv_stack.std()))

The 10-fold crossvalidation RMSE of Stack Regressor is 0.11873 +/- 0.017


In [57]:
#Prediction on real test set using Stack Regressor
stregr.fit(X,y)
model_xgb.fit(X,y)
pred_results_stack =stregr.predict(test_df)
pred_results_stack =np.expm1(pred_results_stack)
pred_results_xgb = model_xgb.predict(test_df)
pred_results_xgb =np.expm1(pred_results_xgb)

ensemble = pred_results_stack*0.30 + pred_results_xgb*0.70
result_df = pd.DataFrame(data={'Id': test_df["Id"].values,
                               'SalePrice': ensemble})
#Create output csv file
result_df.to_csv(data_path+"outputs/stack_lasso_enet_gboost_ensemble_xgb", index=False)